In [22]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc, run_stan_model, remove_chain_dim
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()

import pystan

In [12]:
log_dir = 'log/fabian_binary/20210830_024713_1f_binsim/'
data = load_obj('data', log_dir)

{'random_seed': 2,
 'N': 200,
 'J': 6,
 'K': 1,
 'alpha': array([-0.53,  0.35, -1.4 , -1.4 , -0.96, -2.33]),
 'beta': array([1., 1., 1., 1., 1., 1.]),
 'z': array([-4.16757847e-01, -5.62668272e-02, -2.13619610e+00,  1.64027081e+00,
        -1.79343559e+00, -8.41747366e-01,  5.02881417e-01, -1.24528809e+00,
        -1.05795222e+00, -9.09007615e-01,  5.51454045e-01,  2.29220801e+00,
         4.15393930e-02, -1.11792545e+00,  5.39058321e-01, -5.96159700e-01,
        -1.91304965e-02,  1.17500122e+00, -7.47870949e-01,  9.02525097e-03,
        -8.78107893e-01, -1.56434170e-01,  2.56570452e-01, -9.88779049e-01,
        -3.38821966e-01, -2.36184031e-01, -6.37655012e-01, -1.18761229e+00,
        -1.42121723e+00, -1.53495196e-01, -2.69056960e-01,  2.23136679e+00,
        -2.43476758e+00,  1.12726505e-01,  3.70444537e-01,  1.35963386e+00,
         5.01857207e-01, -8.44213704e-01,  9.76147160e-06,  5.42352572e-01,
        -3.13508197e-01,  7.71011738e-01, -1.86809065e+00,  1.73118467e+00,
        

## Run MCMC

In [20]:

with open(
    'codebase/stancode/models/hmc/model_7.stan',
    'r'
    ) as file:
    model_code = file.read()

    sm = pystan.StanModel(model_code=model_code, verbose=False)
save_obj(sm, 'sm_hmc', log_dir)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cac0b7f4994c30bbd65d089b1cb38865 NOW.


In [24]:
# sm = load_obj('sm', 'log/compiled_models/model8/')
sm = load_obj('sm_hmc', log_dir)

fit_run = run_stan_model(
    data = data.get_stan_data(),
    compiled_model = sm,
    num_samples = 1000, 
    num_warmup = 1000,
    num_chains = 1,
#     initial_values={'beta':[1,1,1,1,1,1]},
    adapt_engaged=True,
    )

param_names = ['beta', 'alpha']

ps = fit_run.extract(
        permuted=False, pars=param_names)

save_obj(ps, 'ps_hmc', log_dir)


In [25]:
ps = load_obj('ps_hmc', log_dir)
nsim = ps['beta'].shape[0]
ps = remove_chain_dim(ps, param_names, num_samples=nsim)


## Run IBIS

In [30]:
# load existing results

ibis = load_obj('particles', log_dir)


In [31]:
gen_model = 0
model_num = 1


for name in ['alpha']:
    samples = np.squeeze(ibis.particles[name])
    w = exp_and_normalise(ibis.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    # print('\nRead Data')
    # print(np.round(exp_data.raw_data[name],2))



Estimate
[-0.4   0.38 -2.   -1.23 -0.82 -2.43]


## Plot Both results

In [32]:
# have to resample particles to get rid of weights
ibis.resample_particles()
particles = ibis.particles


In [33]:
param = 'beta'
df = get_post_df(ps[param])
df['source'] = 'mcmc'
df2 = get_post_df(particles[param])
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))

alt.Chart(...)

In [34]:
param = 'alpha'
df = get_post_df(ps[param])
df['source'] = 'hmc'
df2 = get_post_df(particles[param])
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))


alt.Chart(...)